In [1]:
import torch
from collections import OrderedDict

# 1. Loading the weights file
weight_file_path = "../param/Net_Quant/Net_Quant_lr0.0008_n_steps200_3000best_loss.pth.tar"
loaded_weights = torch.load(weight_file_path)

dest_path = "../param/Net_Quant/Net_Quant_modify.pth.tar"

group = 9
# 2. Get the weights for a specific layer
N = 10
# init_feature is the name of the part to be processed
for x in range(N):
    # N denotes the number of layers of init_feature
    desired_layer_name = f"init_feature.{x}.conv.weight"
    if desired_layer_name in loaded_weights['state_dict']:
        # Extracting weights
        weights_to_modify = loaded_weights['state_dict'][desired_layer_name]
        OFM, IFM_group, k_h, k_w = weights_to_modify.shape
        # Initialize the sparse matrix
        W_sparse = torch.zeros((OFM, IFM_group * group, k_h, k_w), dtype=torch.float)
        num = IFM_group
        # Set up the sparse weight matrix
        for i in range(OFM):
            OFM_group = OFM // group
            n = i // OFM_group
            in_start = n * num
            in_end = (n + 1) * num
            W_sparse[i, in_start:in_end, :, :] = weights_to_modify[i, :, :, :]
        loaded_weights['state_dict'][desired_layer_name] = W_sparse
torch.save(loaded_weights, dest_path)

In [2]:
weight_file_path = dest_path
loaded_weights = torch.load(weight_file_path)

desired_layer_name = f"BuildCost.weight"
if desired_layer_name in loaded_weights['state_dict']:
    print('have')
    # Extracting weights
    weights_to_modify = loaded_weights['state_dict'][desired_layer_name]
    OFM, IFM_group, k_h, k_w = weights_to_modify.shape
    # Initialize the sparse matrix
    W_sparse = torch.zeros((OFM, IFM_group * group, k_h, k_w), dtype=torch.float)
    num = IFM_group
    # Set up the sparse weight matrix
    for i in range(OFM):
        OFM_group = OFM // group
        n = i // OFM_group
        in_start = n * num
        in_end = (n + 1) * num
        W_sparse[i, in_start:in_end, :, :] = weights_to_modify[i, :, :, :]
    loaded_weights['state_dict'][desired_layer_name] = W_sparse
torch.save(loaded_weights, weight_file_path)

have
